In [9]:
import time
import datetime
import requests
import pandas as pd
import os

In [2]:

def fetch_eia_data(api_path, *,api_key=None, frequency=None, facets=None,
                   data_vars=None, start=None, end=None,
                   offset=0, length=5000):
    """Fetch one “page” of data from EIA v2."""
    base = "https://api.eia.gov/v2/"
    url = f"{base}{api_path.rstrip('/')}/data/"
    
    # Build params according to EIA spec
    params = {
        "api_key": api_key,
        "offset": offset,
        "length": length
    }
    if frequency is not None:
        params["frequency"] = frequency
    if data_vars is not None:
        # EIA expects something like data[0]=value, data[1]=other
        for i, dv in enumerate(data_vars):
            params[f"data[{i}]"] = dv
    if facets is not None:
        for facet_key, facet_vals in facets.items():
            for i, val in enumerate(facet_vals):
                print(f'facet_val={val}')
                params[f"facets[{facet_key}][{i}]"] = val
    if start is not None:
        params["start"] = start
    if end is not None:
        params["end"] = end
    
    print("Requesting:", url)
    print("Params:", {p: params[p] for p in params if p!= 'api_key'})
    
    resp = requests.get(url, params=params)
    try:
        resp.raise_for_status()
    except requests.HTTPError as e:
        print("ERROR response status:", resp.status_code)
        print("Response text:", resp.text)
        raise
    
    return resp.json()




In [3]:
def fetch_all_for_series(api_path, **kwargs):
    """Fetch all pages (offset-based) for a series, merging them.

    Returns: 
            pandas.DataFrame: a dataframe containing all rows from the api querey
    """
    all_records = []
    offset = 0
    length = kwargs.get("length")
    
    while True:
        response_json = fetch_eia_data(api_path, offset=offset, **kwargs)
        recs = response_json["response"]["data"]
        if not recs:
            break
        all_records.extend(recs)
        # If fewer than length returned, we've reached the end so we're done'
        if len(recs) < length:
            break
        offset += length
        # sleep to slow requests
        time.sleep(0.2)

    #df 
    
    return pd.DataFrame(all_records)

## Electricity Sales price

In [5]:
api_path = "electricity/retail-sales"

# use this to filter by state or sector e.g.
# facets = {"stateid: ['CA', 'WI'],
#           "sectorid": ['IND', 'RES'] 
#           }
# to get only date for industrial and residential sectors in california and wisconsin
facets=None

# remember to remove api key before committing to repo
API_KEY = "your api key here."

kwargs = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data_vars": ["customers", "price", "revenue", "sales"],  # actual data variables
    "facets": facets,
    "start": "2001-01", #start date - Jan 2001 is as far back as we can go
    "end": "2025-07", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/electricity/retail-sales?frequency=monthly&data=customers;price;revenue;sales;&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop the columns for stateDescription and sectorName since they are redundant
df = df.drop(columns=['stateDescription', 'sectorName'])

Requesting: https://api.eia.gov/v2/electricity/retail-sales/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'customers', 'data[1]': 'price', 'data[2]': 'revenue', 'data[3]': 'sales', 'start': '2001-01', 'end': '2025-07'}
ERROR response status: 403
Response text: {
  "error": {
    "code": "API_KEY_INVALID",
    "message": "An invalid api_key was supplied. Get one at https://api.eia.gov:443"
  }
}


HTTPError: 403 Client Error: Forbidden for url: https://api.eia.gov/v2/electricity/retail-sales/data/?api_key=your+api+key+here.&offset=0&length=5000&frequency=monthly&data%5B0%5D=customers&data%5B1%5D=price&data%5B2%5D=revenue&data%5B3%5D=sales&start=2001-01&end=2025-07

In [ ]:
import os
data_dir = os.path.join(os.pardir, 'datasets')

df.to_csv(os.path.join(data_dir, 'state_electricity_price.csv'), index=False)

# multi-index the dataframe first by month and year then by state

df = df.set_index(["period", "stateid", "sectorid"]).sort_index()

df
#df.to_csv('state_electricity_price.csv')


customers  price   revenue      sales  \
period  stateid sectorid                                         
2001-01 AK      ALL           None   9.97  51.96404  521.03566   
                COM           None    9.7   20.2141  208.49968   
                IND           None   7.05   6.26039    88.7627   
                OTH           None  13.36   2.36091   17.66485   
                RES           None  11.22  23.12865  206.10843   
...                            ...    ...       ...        ...   
2025-07 WY      COM          60181  10.16  46.39331  456.52285   
                IND          12067   8.93  68.35194  765.72477   
                OTH           None   None      None       None   
                RES         289795  14.64  34.96128   238.7893   
                TRA              0      0         0          0   

                              customers-units              price-units  \
period  stateid sectorid                                                 
2001-01 AK      ALL       number of customers  cents per kilowatt-hour   
                COM       number of customers  cents per kilowatt-hour   
                IND       number of customers  cents per kilowatt-hour   
                OTH       number of customers  cents per kilowatt-hour   
                RES       number of customers  cents per kilowatt-hour   
...                                       ...                      ...   
2025-07 WY      COM       number of customers  cents per kilowatt-hour   
                IND       number of customers  cents per kilowatt-hour   
                OTH       number of customers  cents per kilowatt-hour   
                RES       number of customers  cents per kilowatt-hour   
                TRA       number of customers  cents per kilowatt-hour   

                            revenue-units             sales-units  
period  stateid sectorid                                           
2001-01 AK      ALL       million dollars  million kilowatt hours  
                COM       million dollars  million kilowatt hours  
                IND       million dollars  million kilowatt hours  
                OTH       million dollars  million kilowatt hours  
                RES       million dollars  million kilowatt hours  
...                                   ...                     ...  
2025-07 WY      COM       million dollars  million kilowatt hours  
                IND       million dollars  million kilowatt hours  
                OTH       million dollars  million kilowatt hours  
                RES       million dollars  million kilowatt hours  
                TRA       million dollars  million kilowatt hours  

[109740 rows x 8 columns]

## Electricity Generation

In [ ]:
api_path = "electricity/electric-power-operational-data"

states= [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

# sectorid 99 is all sectors
facets= {'sectorid': ['99'], 'location': states} 

# remember to remove api key before committing to repo
API_KEY = "your api key here"

kwargs = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data_vars": ["generation"],  # actual data variables
    "facets": facets,
    "start": "2001-01", #start date - Jan 2001 is as far back as we can go
    "end": "2025-07", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/electricity/retail-sales?frequency=monthly&data=customers;price;revenue;sales;&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop the columns for stateDescription
df = df.drop(columns=['stateDescription'])

#rename location column to stateid to match electricity price data
df = df.rename(columns={'location': 'stateid'})


facet_val=99
facet_val=AL
facet_val=AK
facet_val=AZ
facet_val=AR
facet_val=CA
facet_val=CO
facet_val=CT
facet_val=DE
facet_val=FL
facet_val=GA
facet_val=HI
facet_val=ID
facet_val=IL
facet_val=IN
facet_val=IA
facet_val=KS
facet_val=KY
facet_val=LA
facet_val=ME
facet_val=MD
facet_val=MA
facet_val=MI
facet_val=MN
facet_val=MS
facet_val=MO
facet_val=MT
facet_val=NE
facet_val=NV
facet_val=NH
facet_val=NJ
facet_val=NM
facet_val=NY
facet_val=NC
facet_val=ND
facet_val=OH
facet_val=OK
facet_val=OR
facet_val=PA
facet_val=RI
facet_val=SC
facet_val=SD
facet_val=TN
facet_val=TX
facet_val=UT
facet_val=VT
facet_val=VA
facet_val=WA
facet_val=WV
facet_val=WI
facet_val=WY
Requesting: https://api.eia.gov/v2/electricity/electric-power-operational-data/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'generation', 'facets[sectorid][0]': '99', 'facets[location][0]': 'AL', 'facets[location][1]': 'AK', 'facets[location][2]': 'AZ', 'facets[location][3]': 'AR', 'facets[location][4]

TypeError: can only concatenate str (not "datetime.datetime") to str

In [12]:
print(datetime.datetime.now())

2025-09-27 17:39:40.549127


In [13]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'state_electricity_generation.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,stateid,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,generation,generation-units
0,2022-10,NC,99,All Sectors,MLG,municiapl landfill gas,31.08801,thousand megawatthours
1,2022-10,NC,99,All Sectors,NG,natural gas,4602.89999,thousand megawatthours
2,2022-10,MO,99,All Sectors,COW,all coal products,3596.72909,thousand megawatthours
3,2022-10,MO,99,All Sectors,DFO,distillate fuel oil,10.83543,thousand megawatthours
4,2022-10,MO,99,All Sectors,DPV,estimated small scale solar photovoltaic,44.30459,thousand megawatthours
...,...,...,...,...,...,...,...,...
440873,2003-03,NE,99,All Sectors,MLG,municiapl landfill gas,2.323,thousand megawatthours
440874,2003-03,NE,99,All Sectors,NUC,nuclear,284.14,thousand megawatthours
440875,2003-03,NE,99,All Sectors,OB2,biomass,2.15692,thousand megawatthours
440876,2003-03,NE,99,All Sectors,OOG,other gases,.146,thousand megawatthours


## Other annual state specific data

In [ ]:
api_path = "electricity/state-electricity-profiles/source-disposition"

states= [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

facets= None
data_vars = [
        "facility-direct",
        "net-interstate-trade",
        "total-elect-indust",
        "total-international-exports",
        "total-international-imports",
        "total-net-generation"
    ]

# remember to remove api key before committing to repo
API_KEY = "your api key here"

kwargs = {
    "api_key": API_KEY,
    "frequency": "annual", #only annual data available from this db
    "data_vars": data_vars,  # actual data variables
    "facets": facets,
    "start": "2001", #start date - Jan 2001 is as far back as we can go
    "end": "2023", #end date- 2023 is as far as we can go
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/electricity/state-electricity-profiles/source-disposition?frequency=annual&data=facility-direct;net-interstate-trade;total-elect-indust;total-international-exports;total-international-imports;total-net-generation;&start=2001&end=2023&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop the columns for stateDescription
df = df.drop(columns=['stateDescription'])

#rename state column to stateid to match electricity price data
df = df.rename(columns={'state': 'stateid'})

print(datetime.datetime.now())

Requesting: https://api.eia.gov/v2/electricity/state-electricity-profiles/source-disposition/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'annual', 'data[0]': 'facility-direct', 'data[1]': 'net-interstate-trade', 'data[2]': 'total-elect-indust', 'data[3]': 'total-international-exports', 'data[4]': 'total-international-imports', 'data[5]': 'total-net-generation', 'start': '2001', 'end': '2023'}
2025-09-27 17:57:14.685240


In [16]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'state_electricity_annual_data.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,stateid,facility-direct,net-interstate-trade,total-elect-indust,total-international-exports,total-international-imports,total-net-generation,facility-direct-units,net-interstate-trade-units,total-elect-indust-units,total-international-exports-units,total-international-imports-units,total-net-generation-units
0,2013,MN,942126,-15273728,68644103,262439,8179786,51296988,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1,2017,MO,None,3165496,76461419,0,0,84606731,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
2,2015,MO,None,-3468206,81504081,0,0,83640067,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
3,2004,MO,19684,4488629,74054296,6276,0,87632910,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
4,2006,MO,19690,433931,82015230,10,2975,91686343,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,2022,CO,179780,-1909941,56763041,0,0,58044009,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1192,2002,CO,141066,-5681503,46077762,0,6707,45600388,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1193,2023,CO,175842,-1018485,55565819,0,0,57541720,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1194,2007,CO,248794,-1726737,51299156,1679,1268,53907492,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours


## Coal

### Export and Import prices

In [ ]:
api_path = "coal/exports-imports-quantity-price"

states= [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

facets= {'countryId': ['TOT'], 'customsDistrictId': ['TOT']}
data_vars = ['price']

# remember to remove api key before committing to repo
API_KEY = "your api key here"

kwargs = {
    "api_key": API_KEY,
    "frequency": "quarterly", #only quarterly data available from this db
    "data_vars": data_vars,  # actual data variables
    "facets": facets,
    "start": "2001-Q1", #start date
    "end": "2024-Q1", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/coal/exports-imports-quantity-price?frequency=quarterly&data=price;&facets=countryId;customsDistrictId;&countryId=TOT;&customsDistrictId=TOT;&start=2001-01&end=2024-01&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop theses columns because we only look at total imports (not broken down by exporting country and location of import)
df = df.drop(columns=['countryId', 'countryDescription', 'customsDistrictId', 'customsDistrictDescription'])

print(datetime.datetime.now())

facet_val=TOT
facet_val=TOT
Requesting: https://api.eia.gov/v2/coal/exports-imports-quantity-price/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'quarterly', 'data[0]': 'price', 'facets[countryId][0]': 'TOT', 'facets[customsDistrictId][0]': 'TOT', 'start': '2001-Q1', 'end': '2024-Q1'}
2025-09-27 18:11:22.930983


In [19]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'quarterly_coal_import_price.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,exportImportType,coalRankId,coalRankDescription,price,price-units
0,2008-Q1,Exports,MET,Metallurgical,98.9,dollars per short ton
1,2010-Q3,Imports,MET,Metallurgical,204.71,dollars per short ton
2,2004-Q2,Exports,MET,Metallurgical,65.32,dollars per short ton
3,2004-Q1,Exports,MET,Metallurgical,54.1,dollars per short ton
4,2004-Q1,Imports,MET,Metallurgical,48.42,dollars per short ton
...,...,...,...,...,...,...
739,2023-Q1,Imports,STM,Steam Coal,129.34,dollars per short ton
740,2023-Q3,Imports,STM,Steam Coal,114.71,dollars per short ton
741,2023-Q1,Exports,STM,Steam Coal,119.62,dollars per short ton
742,2023-Q3,Exports,STM,Steam Coal,85.74,dollars per short ton
